In [ ]:
import pandas as pd
import json
from collections import defaultdict
import re
from datetime import datetime, timedelta

In [ ]:
datos = pd.read_json('traces_Articoding_Escolapias.json')

#### Con este bucle obtenemos los datos sobre la inicialización de todos los niveles jugados

In [ ]:
JSONFile = open('traces_Articoding_Escolapias.json')
rawData = json.load(JSONFile)

erLevel = re.compile(r'\blevel$\b')
erInitialized = re.compile(r'\binitialized$\b')
erCompleted = re.compile(r'\bcompleted$\b')
erIdLevel = re.compile(r'/')
data = defaultdict(defaultdict)
anomalias = defaultdict()

for evento in rawData:
    verb = evento["verb"]["id"]
    obj = evento["object"]["definition"]["type"]
    if erLevel.search(obj) : #Si el objeto de la acción es un nivel
        name = evento["actor"]["name"]
        levelCode = erIdLevel.split(evento["object"]["id"])[-1]
        timestamp = evento["timestamp"]
        if erInitialized.search(verb): #Si a la acción es inicio o reinicio
            if "result" in evento: #Significa que ha iniciado el nivel desde el menu
                if levelCode in data[name]:
                    data[name][levelCode].append({"ini" : timestamp, "fin" : ""})
                else:
                    data[name][levelCode] = [{"ini" : timestamp, "fin" : ""}]
            #else: #-> sería cuando un nivel es reiniciado, por ahora no vamos a tenerlo en cuenta
        elif erCompleted.search(verb):
            if evento["result"]["success"]: #Significa que el nivel se ha completado con éxito
                if levelCode in data[name]:
                    data[name][levelCode][-1]["fin"] = timestamp
                else: #Significa que hay un evento de finalizar que no tiene un evento de inicio
                    anomalias[evento["_id"]] = evento
            else:
                if levelCode in data[name]:
                    data[name][levelCode][-1]["fin"] = "Nivel abortado" #Porque el jugador sale al menú o cierra el juego
                else: #Significa que hay un evento de finalizar que no tiene un evento de inicio
                    anomalias[evento["_id"]] = evento
                    
                    
JSONFile.close()
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.DataFrame(data)